Etape préliminaire 

Avant de commencer le projet, il faut installer les packages dont nous allons avoir besoin

In [1]:
import requests
import pandas as pd
import zipfile
import os

Etape 1 : Importation des données

Afin que cette étape se déroule bien, il faut récupérer la base DS_BPE_2024_data.csv préalablement importée sur S3.

In [3]:
# =========================
# ÉTAPE 1 : DONNÉES ÉLECTORALES
# =========================

url_legisl = "https://www.data.gouv.fr/api/1/datasets/r/ab337c6f-e7e8-4981-843c-45052b71096b"
path_legisl = "/home/onyxia/work/Projet-Python-S3/code/results_legisl.xlsx"

# Téléchargement du fichier Excel
response = requests.get(url_legisl)
response.raise_for_status()

with open(path_legisl, "wb") as f:
    f.write(response.content)

# Lecture du fichier Excel
df_legisl = pd.read_excel(path_legisl, sheet_name=0)

print("Le dataframe df_df_legisl peut être traité comme un data frame classique avec pandas")


# =========================
# ÉTAPE 2 : DONNÉES INFRASTRUCTURES (S3 SSPCloud)
# =========================

# Chemin S3 du fichier
path_infra_s3 = "s3://hantier/DS_BPE_2024_data.csv"

# Lecture directe depuis S3
df_infrastruct = pd.read_csv("s3://hantier/DS_BPE_2024_data.csv", sep=';')  # le séparateur utilisé est un ;
print("Le dataframe df_infrastruct peut être traité comme un data frame classique avec pandas")

Le dataframe df_df_legisl peut être traité comme un data frame classique avec pandas
Le dataframe df_infrastruct peut être traité comme un data frame classique avec pandas


/tmp/ipykernel_9858/358226819.py:29: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_infrastruct = pd.read_csv("s3://hantier/DS_BPE_2024_data.csv", sep=';')  # le séparateur utilisé est un ;


Etape 2 : Préparation du dataframe que nous allons utiliser

Ici, la variable nuance candidat prend en valeur un code qui correspond au parti politique du candidat. Nous allons remplacer cette variable par une variable numérique qui attribuera un score allant de - 2 (extrême gauche) à 2 (extrême droite) ce qui nous permettra ensuite en faisant une moyenne pondérée des trois candidats les plus plébiscités afin, ensuite de placer les infrastructures sur un axes gauche-droite du bord politique de la commune

Les variables nuance prennent les valeurs suivantes : DIV, DSV, DVC, DVD, DVG, ECO, ENS, EXD, EXG, HOR, LR, REG, REC, RN, SOC, UDI, UG, UXD, COM, UDI

On crée un dataframe (bord_pol) qui permet de rapidement faire le changement

In [4]:
df_legisl_1 = df_legisl[["Code département", "Libellé département", "Code commune", "Libellé commune",  "% Votants", "Nuance candidat 1", "% Voix/exprimés 1", "Nuance candidat 2", "% Voix/exprimés 2", "Nuance candidat 3", "% Voix/exprimés 3"]]
print("Informations sur le dataframe df_legisl_1")
print(df_legisl_1.columns)
print(df_legisl_1.size)
print(df_legisl_1.head())

bord_pol = [["DIV", 0], 
["DSV", 0], 
["DVC", 0], 
["DVD", 0], 
["DVG", 0], 
["ECO", -1], 
["ENS", 0], 
["EXD", 2], 
["EXG", -2], 
["HOR", 0], 
["LR", 1], 
["REG", 0], 
["REC", 0], 
["RN", 2], 
["SOC", 0], 
["UDI", 0], 
["UG", 0], 
["UXD", 0], 
["COM", 2], 
["UDI", 0]]


Informations sur le dataframe df_legisl_1
Index(['Code département', 'Libellé département', 'Code commune',
       'Libellé commune', '% Votants', 'Nuance candidat 1',
       '% Voix/exprimés 1', 'Nuance candidat 2', '% Voix/exprimés 2',
       'Nuance candidat 3', '% Voix/exprimés 3'],
      dtype='object')
387552
  Code département Libellé département Code commune          Libellé commune  \
0                1                 Ain         1001  L'Abergement-Clémenciat   
1                1                 Ain         1002    L'Abergement-de-Varey   
2                1                 Ain         1004        Ambérieu-en-Bugey   
3                1                 Ain         1005      Ambérieux-en-Dombes   
4                1                 Ain         1006                  Ambléon   

  % Votants Nuance candidat 1 % Voix/exprimés 1 Nuance candidat 2  \
0    74,32%                UG            17,44%               DIV   
1    78,07%               ECO             1,17%               EX